In [34]:
import matplotlib
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import r2_score

In [35]:
df = pd.read_csv('data1011train.csv',encoding='UTF-8-Sig')
x = torch.tensor(df.iloc[:,[10,60,74,78,89,90,91,93,100,117,124,129,131,133,150,153,154,161,162,163,164,165,166,167,168]].to_numpy()).to(torch.float32)
y = torch.tensor(df.iloc[:,-1].to_numpy()).to(torch.float32)
x.shape

torch.Size([2950, 25])

In [36]:
class stockDataSetter(Dataset):
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
            self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [37]:
Batchsize = (x.shape[0]//10)*8
train_set = stockDataSetter(x, y)
train_loader = DataLoader(train_set, Batchsize,shuffle=True, pin_memory=True)

In [38]:
import torch
import torch.nn.functional as F   

class My_model(torch.nn.Module): 
    def __init__(self, input_dim):
        super(My_model, self).__init__()
        self.layers = torch.nn.Sequential(
            nn.Linear(input_dim, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
            )
        
    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

net = My_model(input_dim=x.shape[1])
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.RAdam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-5, weight_decay=0, foreach=True)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False, foreach=None, maximize=False, capturable=False)
loss_function = torch.nn.MSELoss(reduction='mean')

print(net)  # net 的结构

My_model(
  (layers): Sequential(
    (0): Linear(in_features=25, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=1, bias=True)
  )
)


In [39]:
#訓練
# checkpoint = torch.load('model_state_dict1.pt')
# net.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']
net = net.train()

best_loss = math.inf
patience_bestloss = 0

losslist = []
for t in range(1000000):
          
    x, y = next(iter(train_loader))
    prediction = net(x).to(device)
    loss = loss_function(prediction,y).to(device)
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()


    # 接着上面来
    if t % 50 == 0:

        print('================================================')
        print(f'第:{t}輪', 'loss: ',loss.data.numpy())
        print(f'目前最佳loss: {best_loss}')
        print('patience_bestloss(最大1000): ',patience_bestloss )
        print('================================================')
    
    losslist.append(loss.data.numpy())
    if len(losslist)>50:
        losslist = losslist[1:]
    
    avgloss = sum(losslist)/len(losslist
                               )
    if avgloss<best_loss:
        patience_bestloss = 0
        best_loss = avgloss
        torch.save(net, 'Save_File1.pth')
        torch.save({
            'epoch': t,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, 'model_state_dict1.pt')

    else:
        patience_bestloss += 1
    
    if patience_bestloss >= 5000:
        print('loss已經訓練1000個batch未下降')
        break
    

第:0輪 loss:  7424795.0
目前最佳loss: inf
patience_bestloss(最大1000):  0
第:50輪 loss:  45386.098
目前最佳loss: 870250.991640625
patience_bestloss(最大1000):  0
第:100輪 loss:  28692.53
目前最佳loss: 33043.755859375
patience_bestloss(最大1000):  0
第:150輪 loss:  28559.982
目前最佳loss: 28456.4915234375
patience_bestloss(最大1000):  0
第:200輪 loss:  26400.533
目前最佳loss: 26678.3437109375
patience_bestloss(最大1000):  0
第:250輪 loss:  23686.666
目前最佳loss: 25012.8096875
patience_bestloss(最大1000):  0
第:300輪 loss:  23177.742
目前最佳loss: 24210.718359375
patience_bestloss(最大1000):  1
第:350輪 loss:  23165.297
目前最佳loss: 23543.246015625
patience_bestloss(最大1000):  8
第:400輪 loss:  23265.287
目前最佳loss: 23049.6257421875
patience_bestloss(最大1000):  0
第:450輪 loss:  23044.756
目前最佳loss: 22461.1506640625
patience_bestloss(最大1000):  0
第:500輪 loss:  21868.69
目前最佳loss: 22269.3496484375
patience_bestloss(最大1000):  3
第:550輪 loss:  22542.688
目前最佳loss: 21858.5465625
patience_bestloss(最大1000):  0
第:600輪 loss:  22162.904
目前最佳loss: 21638.5614453125
pati

KeyboardInterrupt: 

In [ ]:
#檢查
checkpoint = torch.load('model_state_dict1.pt')
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
net = net.eval()

prediction = net(x).to(device)
loss = loss_function(prediction,y).to(device)

pred,actual = pd.DataFrame(prediction.data.numpy()), pd.DataFrame(y.data.numpy())
compare = pd.DataFrame(columns=['actual','predict'])
compare['actual'],compare['predict'] = y.data.numpy(), prediction.data.numpy()
compare = compare.T

In [ ]:
ax = plt.gca()
print('loss: ',loss.data.numpy())
compare.T.plot(kind='line',y ='predict',ax=ax,color='black')
compare.T.plot(kind='line',y ='actual',ax=ax,color='red')
plt.show()
compare
# Display all model layer weights
# for name, para in net.named_parameters():
#     print('{}: {}'.format(name, para.shape))

In [ ]:
r2_score(compare.T['actual'],compare.T['predict'])